In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
commercial_bank = [
    '국민은행', '신한은행', '우리은행', '하나은행', '외환은행', '아이엠뱅크'
    # '씨티은행', '스탠다드차타드은행'
] # https://www.bok.or.kr/portal/main/contents.do?menuNo=200580


In [3]:
def info_issue_trans(issue: pd.DataFrame, trans:pd.DataFrame, columns:list[str]) -> None:
    for v in (issue, trans):
        print()
        print(v.shape)
        print(v.columns)
        print("issue_date:", min(v["발행일"]), max(v["발행일"]))
        print("mat_date:", min(v["만기일"]), max(v["만기일"]))
        for c in columns:
            print(c, v[c].unique())

In [4]:
def info_merge(merge: pd.DataFrame) -> None:
    print(merge.shape)
    print(merge.columns)
    print(merge["_merge"].value_counts())
    print(merge["_merge"].value_counts() / merge.shape[0]* 100)

# Kofia

In [6]:
trans = pd.read_csv("kofia/trans_detail_CD.csv",
                    dtype={"발행일": int, "만기일": int, "발행금액(억원)": float})
trans.loc[trans["발행기관"].str.contains("하나 은행", na=False), "발행기관"] = "하나은행"
print(trans.shape)

issue = pd.read_csv("kofia/issue_CD.csv",
                    dtype={"발행일": int, "만기일": int, "발행금액(억원)": float}
                    ).drop_duplicates(
).groupby(
    ["발행일", "만기일", "통화구분", "발행기관", "발행금리(%)", "만기"]
).agg(
    {"발행금액(억원)": "sum"}
).sort_values(
    "발행금액(억원)", ascending=False
).reset_index()

issue.loc[issue["발행기관"].str.contains("하나은행"), "발행기관"] = "하나은행"

(70952, 12)


In [7]:
info_issue_trans(issue, trans, ["통화구분", "발행기관"])


(2813, 7)
Index(['발행일', '만기일', '통화구분', '발행기관', '발행금리(%)', '만기', '발행금액(억원)'], dtype='object')
issue_date: 20121023 20231229
mat_date: 20130107 20250512
통화구분 ['원화' '달러화']
발행기관 ['NH농협은행' '국민은행' '농협' '우리은행' '하나은행' '한국산업은행' '스탠다드차타드은행' '신한은행' 'KDB산업은행'
 '씨티은행' '중국농업은행' '수협은행' '산업은행' '경남은행' '기업은행' '중국건설은행' '외환은행' '부산은행' '수협'
 '전북은행' '대구은행' '중국은행' '중국공상은행' '광주은행' '대화은행' '제주은행' '중국광대은행' 'OCBC은행 서울지점'
 '바클레이즈은행']

(70952, 12)
Index(['거래일자', '시간', '발행기관', '발행일', '만기일', '잔존기간', '수익률(%)', '거래량(억원)',
       '거래대금(억원)', '통화구분', '거래구분', '정정여부'],
      dtype='object')
issue_date: 20080929 20231229
mat_date: 20100107 20250512
통화구분 ['원화' '달러화']
발행기관 ['NH농협은행' '하나은행' '씨티은행' '한국산업은행' '국민은행' '수협은행' '기업은행' '스탠다드차타드은행' '우리은행'
 '신한은행' '아이엠뱅크' '부산은행' nan '제주은행' 'HSBC은행' '중국공상은행' '중국은행' '광주은행' '중국농업은행'
 '중국건설은행' '경남은행' '전북은행' '중국광대은행' 'OCBC은행 서울지점' '대화은행']


In [8]:
issue.head()

,발행일,만기일,통화구분,발행기관,발행금리(%),만기,발행금액(억원)
0,20200106,20200406,원화,NH농협은행,1.49000,91,10000.00000
1,20200421,20200921,원화,NH농협은행,1.20000,153,8100.00000
2,20210513,20211112,원화,국민은행,0.67000,183,8000.00000
3,20191111,20201110,원화,NH농협은행,1.71000,365,7100.00000
4,20220826,20221124,원화,국민은행,2.92000,90,7100.00000


In [9]:
issue.describe()

,발행일,만기일,발행금리(%),만기,발행금액(억원)
count,2813.00000,2813.00000,2813.00000,2813.00000,2813.00000
mean,20188421.27622,20193149.86065,2.30697,168.67579,1098.08745
std,33692.65935,35373.85059,1.10219,113.38875,927.01074
min,20121023.00000,20130107.00000,0.55000,27.00000,50.00000
25%,20160707.00000,20161004.00000,1.52000,91.00000,500.00000
50%,20190718.00000,20200224.00000,1.88000,122.00000,1000.00000
75%,20220722.00000,20230113.00000,3.07000,185.00000,1450.00000
max,20231229.00000,20250512.00000,5.60000,1249.00000,10000.00000


In [10]:
trans.head()

,거래일자,시간,발행기관,발행일,만기일,잔존기간,수익률(%),거래량(억원),거래대금(억원),통화구분,거래구분,정정여부
0,20100104,31129,NH농협은행,20091117,20100517,133,3.17000,100,99,원화,매도,NaN
1,20100104,31129,하나은행,20091201,20100601,148,3.49000,100,98,원화,매수,NaN
2,20100104,31129,하나은행,20091201,20100601,148,3.49000,100,98,원화,매도,NaN
3,20100104,31129,하나은행,20091201,20100601,148,3.49000,100,98,원화,매도,NaN
4,20100104,31129,씨티은행,20100104,20100305,60,2.81000,100,99,원화,매수,NaN


In [11]:
trans.describe()

,거래일자,시간,발행일,만기일,잔존기간,수익률(%),거래량(억원),거래대금(억원)
count,70952.00000,70952.00000,70952.00000,70952.00000,70952.00000,70952.00000,70952.00000,70952.00000
mean,20189645.86071,124352.80918,20187725.11491,20192840.34901,131.97703,2.69441,154.51626,1404777.45553
std,41083.23183,34604.10824,40608.18151,41956.71242,115.26475,75.08931,199.20199,139998290.24566
min,20100104.00000,31129.00000,20080929.00000,20100107.00000,-304.00000,0.30000,0.00000,0.00000
25%,20160202.00000,105230.00000,20160125.00000,20160429.00000,45.00000,1.51000,50.00000,50.00000
50%,20210202.00000,133547.00000,20201106.00000,20210602.00000,91.00000,2.27000,100.00000,99.00000
75%,20221110.00000,144429.00000,20220727.00000,20230406.00000,182.00000,3.52000,100.00000,100.00000
max,20231229.00000,230526.00000,20231229.00000,20250512.00000,1159.00000,20000.00000,4800.00000,19939199098.00000


In [12]:
# 시중은행만 보면 unique

issue_x = issue[issue["발행기관"].isin(commercial_bank)]


t = issue_x[["발행일", "만기일", "통화구분", "발행기관"]].value_counts()

temp = pd.merge(
    t[t > 1], issue,
    on=["발행일", "만기일", "통화구분", "발행기관"],
    how="inner"
)

temp.shape

(0, 8)

In [13]:
merge = pd.merge(
    trans, issue, on=["발행일", "만기일", "통화구분", "발행기관",],
    how="left", validate="m:1", indicator=True)

merge.to_csv("kofia/kofia_trans_issue_cd.csv", index=False)
info_merge(merge)

(70952, 16)
Index(['거래일자', '시간', '발행기관', '발행일', '만기일', '잔존기간', '수익률(%)', '거래량(억원)',
       '거래대금(억원)', '통화구분', '거래구분', '정정여부', '발행금리(%)', '만기', '발행금액(억원)',
       '_merge'],
      dtype='object')
_merge
both          55530
left_only     15422
right_only        0
Name: count, dtype: int64
_merge
both         78.26418
left_only    21.73582
right_only    0.00000
Name: count, dtype: float64


In [219]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70952 entries, 0 to 70951
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   거래일자      70952 non-null  int64   
 1   시간        70952 non-null  int64   
 2   발행기관      70251 non-null  object  
 3   발행일       70952 non-null  int32   
 4   만기일       70952 non-null  int32   
 5   잔존기간      70952 non-null  int64   
 6   수익률(%)    70952 non-null  float64 
 7   거래량(억원)   70952 non-null  int64   
 8   거래대금(억원)  70952 non-null  int64   
 9   통화구분      70952 non-null  object  
 10  거래구분      70952 non-null  object  
 11  정정여부      1726 non-null   object  
 12  발행금리(%)   55530 non-null  float64 
 13  만기        55530 non-null  float64 
 14  발행금액(억원)  55530 non-null  float64 
 15  _merge    70952 non-null  category
dtypes: category(1), float64(4), int32(2), int64(5), object(4)
memory usage: 7.6+ MB


In [220]:
t = merge[merge["_merge"] == "both"]
tt = t[t["발행기관"].isin(commercial_bank)]
print(
    "Matched Commerical Banks of not matched: ",
    tt.shape, tt.shape[0] / t.shape[0]*100
)

t = merge[merge["_merge"] == "left_only"]
tt = t[t["발행기관"].isin(commercial_bank)]
print(
    "Only Commerical Banks of not matched: ",
    tt.shape, tt.shape[0] / t.shape[0]*100
)

Matched Commerical Banks of not matched:  (36203, 16) 65.1953898793445
Only Commerical Banks of not matched:  (4964, 16) 32.187783685643886


In [221]:
merge.head()

,거래일자,시간,발행기관,발행일,만기일,잔존기간,수익률(%),거래량(억원),거래대금(억원),통화구분,거래구분,정정여부,발행금리(%),만기,발행금액(억원),_merge
0,20100104,31129,NH농협은행,20091117,20100517,133,3.17000,100,99,원화,매도,NaN,NaN,NaN,NaN,left_only
1,20100104,31129,하나은행,20091201,20100601,148,3.49000,100,98,원화,매수,NaN,NaN,NaN,NaN,left_only
2,20100104,31129,하나은행,20091201,20100601,148,3.49000,100,98,원화,매도,NaN,NaN,NaN,NaN,left_only
3,20100104,31129,하나은행,20091201,20100601,148,3.49000,100,98,원화,매도,NaN,NaN,NaN,NaN,left_only
4,20100104,31129,씨티은행,20100104,20100305,60,2.81000,100,99,원화,매수,NaN,NaN,NaN,NaN,left_only


In [222]:
print(issue.shape, merge.shape)

issue = issue[issue["발행기관"].isin(commercial_bank)]
merge = merge[(merge["발행기관"].isin(commercial_bank)) & (merge["통화구분"]=="원화")]
info_issue_trans(issue, merge, ["통화구분"])

(2813, 7) (70952, 16)

(1162, 7)
Index(['발행일', '만기일', '통화구분', '발행기관', '발행금리(%)', '만기', '발행금액(억원)'], dtype='object')
issue_date: 20121024 20231226
mat_date: 20130107 20250512
통화구분 ['원화']

(41158, 16)
Index(['거래일자', '시간', '발행기관', '발행일', '만기일', '잔존기간', '수익률(%)', '거래량(억원)',
       '거래대금(억원)', '통화구분', '거래구분', '정정여부', '발행금리(%)', '만기', '발행금액(억원)',
       '_merge'],
      dtype='object')
issue_date: 20080929 20231226
mat_date: 20100107 20250512
통화구분 ['원화']


# Seibro

In [223]:
# 같은 종목번호에 대해 발행금액이 다른 경우가 있음. 합산하여 사용해도 됨

issue = pd.read_csv( 
    "seibro/seibro_cd_issue.csv").drop(columns=["NUM", "증권구분", "INPUT_NOS", "date"])
print(issue.shape, issue["종목번호"].nunique())

issue = pd.merge(
    issue, issue.groupby(
        ["종목번호"]).agg({"발행금액": "sum"}).sort_values(
        "발행금액", ascending=False
    ).reset_index(), on="종목번호", how="left"
).drop(columns=["발행금액_x"]).rename(columns={"발행금액_y": "발행금액"}).drop_duplicates()
print(issue.shape, issue["종목번호"].nunique())


print(issue["발행인유형"].unique(), issue["발행인유형"].isna().sum())

(10024, 15) 3593
(3593, 15) 3593
['특수은행' '지방은행' '외은지점' '시중은행'] 0


In [224]:
trans = pd.read_csv("seibro/seibro_cd_trans.csv").drop(
    columns=["NUM", "증권구분", "date"])

info_issue_trans(issue, trans, ["통화"])


(3593, 15)
Index(['기준일자', 'ISSUCO_INDTP_CLSF_NO', '발행인유형', 'GOODS_CREDITRT_CD',
       'GOODS_CREDITRT_CD_NM', '통화', '할인율', 'SHORTM_FNCEGD_CD', '종목번호', '종목명',
       '발행일', '만기일', 'GOODS_DAYOBJ_CD', '일물', '발행금액'],
      dtype='object')
issue_date: 20120112 20241115
mat_date: 20120213 20251113
통화 ['KRW' 'USD' 'CNY']

(65146, 15)
Index(['기준일자', 'SELLER_INDTP_TPCD', '매도유형', 'BUYER_INDTP_TPCD', '매수유형', '통화',
       'SETL_AMT', '금리', 'SHORTM_FNCEGD_CD', '종목번호', '종목명', '발행일', '만기일',
       'GOODS_LEF_XPIR_TPCD', '잔존만기'],
      dtype='object')
issue_date: 20081111 20241114
mat_date: 20120118 20251113
통화 ['KRW']


In [225]:
issue.head()

,기준일자,ISSUCO_INDTP_CLSF_NO,발행인유형,GOODS_CREDITRT_CD,GOODS_CREDITRT_CD_NM,통화,할인율,SHORTM_FNCEGD_CD,종목번호,종목명,발행일,만기일,GOODS_DAYOBJ_CD,일물,발행금액
0,20120112,9999,특수은행,1,무위험,KRW,3.53700,13,KRZE0070121C,수협 여의도증권타운지점 20120112-96-1,20120112,20120417,5,3월이상 1년미만,100000000000
3,20120112,9999,지방은행,1,무위험,KRW,3.49000,13,KRZE0310321C,대구 자금부(서울) 20120112-60-3,20120112,20120312,4,1월이상 3월미만,50000000000
4,20120113,9999,외은지점,1,무위험,KRW,3.40000,13,KRZE0510121D,ING Bank 서울(지) 20120113-31-1,20120113,20120213,4,1월이상 3월미만,20000000000
5,20120117,9999,특수은행,1,무위험,KRW,3.53800,13,KRZE0070121H,수협 여의도증권타운지점 20120117-94-1,20120117,20120420,5,3월이상 1년미만,50000000000
7,20120117,9999,특수은행,1,무위험,KRW,3.52300,13,KRZE0070221H,수협 여의도증권타운지점 20120117-80-2,20120117,20120406,4,1월이상 3월미만,50000000000


In [226]:
issue.describe()

,기준일자,ISSUCO_INDTP_CLSF_NO,GOODS_CREDITRT_CD,할인율,SHORTM_FNCEGD_CD,발행일,만기일,GOODS_DAYOBJ_CD,발행금액
count,3593.00000,3593.00000,3593.00000,3593.00000,3593.00000,3593.00000,3593.00000,3593.00000,3593.00000
mean,20192751.75369,3717.34539,1.00000,2.45633,13.00000,20192751.75369,20197494.44531,4.85500,100902847102.73866
std,37448.32481,4759.08350,0.00000,1.05945,0.00000,37448.32481,39096.12206,0.58647,89775255264.79402
min,20120112.00000,101.00000,1.00000,0.00000,13.00000,20120112.00000,20120213.00000,1.00000,513324.00000
25%,20160816.00000,101.00000,1.00000,1.59200,13.00000,20160816.00000,20161202.00000,5.00000,40000000000.00000
50%,20200612.00000,102.00000,1.00000,2.13349,13.00000,20200612.00000,20201218.00000,5.00000,80000000000.00000
75%,20230227.00000,9999.00000,1.00000,3.50000,13.00000,20230227.00000,20231002.00000,5.00000,130000000000.00000
max,20241115.00000,9999.00000,1.00000,5.26600,13.00000,20241115.00000,20251113.00000,6.00000,1000000000000.00000


In [227]:
trans.head()

,기준일자,SELLER_INDTP_TPCD,매도유형,BUYER_INDTP_TPCD,매수유형,통화,SETL_AMT,금리,SHORTM_FNCEGD_CD,종목번호,종목명,발행일,만기일,GOODS_LEF_XPIR_TPCD,잔존만기
0,20120102,101,증권사,101,증권사,KRW,9910571348,3.58000,13,KRZE007011BQ,수협 여의도증권타운지점 20111124-131-1,20111124,20120403,5.00000,3월이상 1년미만
1,20120102,101,증권사,204,집합투자,KRW,9969408936,3.50000,13,KRZE027011AS,한국씨티 자금부 20111026-100-1,20111026,20120203,4.00000,1월이상 3월미만
2,20120102,101,증권사,204,집합투자,KRW,9925666277,3.55000,13,KRZE007021B8,수협 여의도증권타운지점 20111108-132-2,20111108,20120319,4.00000,1월이상 3월미만
3,20120102,101,증권사,204,집합투자,KRW,9910571348,3.58000,13,KRZE007011BQ,수협 여의도증권타운지점 20111124-131-1,20111124,20120403,5.00000,3월이상 1년미만
4,20120102,101,증권사,204,집합투자,KRW,9984681037,3.50000,13,KRZE007011AB,수협 여의도증권타운지점 20111011-99-1,20111011,20120118,3.00000,7일이상 1월미만


In [228]:
trans.describe()

,기준일자,SELLER_INDTP_TPCD,BUYER_INDTP_TPCD,SETL_AMT,금리,SHORTM_FNCEGD_CD,발행일,만기일,GOODS_LEF_XPIR_TPCD
count,65146.00000,65146.00000,65146.00000,65146.00000,65146.00000,65146.00000,65146.00000,65146.00000,65134.00000
mean,20203653.71403,142.46586,200.57155,10445909199.19817,2.54694,13.00000,20201335.48692,20206571.70139,4.31002
std,35801.24828,115.69644,127.13447,10652317861.93780,1.13197,0.00000,35014.74246,36462.59469,1.00070
min,20120102.00000,101.00000,101.00000,210289087.00000,0.00000,13.00000,20081111.00000,20120118.00000,1.00000
25%,20181018.25000,101.00000,101.00000,4973237171.00000,1.55000,13.00000,20180829.00000,20190227.00000,4.00000
50%,20220211.00000,101.00000,204.00000,9918206049.00000,2.65000,13.00000,20211111.00000,20220630.00000,5.00000
75%,20230824.00000,101.00000,204.00000,9979826395.75000,3.60000,13.00000,20230601.00000,20231201.00000,5.00000
max,20241114.00000,901.00000,901.00000,296706557160.00000,5.60000,13.00000,20241114.00000,20251113.00000,6.00000


In [229]:
merge = pd.merge(
    trans, issue, on=["종목번호"], how="left", validate="m:1", indicator=True
).drop(columns=["기준일자_y", "통화_y", "종목명_y", "발행일_y", "만기일_y", "SHORTM_FNCEGD_CD_y"]
       ).rename(columns={
           "기준일자_x": "기준일자", "통화_x": "통화", "종목명_x": "종목명", "발행일_x": "발행일", "만기일_x": "만기일", "SHORTM_FNCEGD_CD_x": "SHORTM_FNCEGD_CD"
       })


info_merge(merge)

(65146, 24)
Index(['기준일자', 'SELLER_INDTP_TPCD', '매도유형', 'BUYER_INDTP_TPCD', '매수유형', '통화',
       'SETL_AMT', '금리', 'SHORTM_FNCEGD_CD', '종목번호', '종목명', '발행일', '만기일',
       'GOODS_LEF_XPIR_TPCD', '잔존만기', 'ISSUCO_INDTP_CLSF_NO', '발행인유형',
       'GOODS_CREDITRT_CD', 'GOODS_CREDITRT_CD_NM', '할인율', 'GOODS_DAYOBJ_CD',
       '일물', '발행금액', '_merge'],
      dtype='object')
_merge
both          65060
left_only        86
right_only        0
Name: count, dtype: int64
_merge
both         99.86799
left_only     0.13201
right_only    0.00000
Name: count, dtype: float64


In [230]:
merge = merge[[
    "기준일자", "종목번호", "종목명", "잔존만기", "매도유형", "매수유형", 
    "발행일", "만기일", "통화", "발행금액", "발행인유형", "금리", "할인율", "일물", "_merge"
]]
merge.to_csv("seibro/seibro_trans_issue_cd.csv", index=False)   
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65146 entries, 0 to 65145
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   기준일자    65146 non-null  int64   
 1   종목번호    65146 non-null  object  
 2   종목명     65146 non-null  object  
 3   잔존만기    65134 non-null  object  
 4   매도유형    65146 non-null  object  
 5   매수유형    65146 non-null  object  
 6   발행일     65146 non-null  int64   
 7   만기일     65146 non-null  int64   
 8   통화      65146 non-null  object  
 9   발행금액    65060 non-null  float64 
 10  발행인유형   65060 non-null  object  
 11  금리      65146 non-null  float64 
 12  할인율     65060 non-null  float64 
 13  일물      65060 non-null  object  
 14  _merge  65146 non-null  category
dtypes: category(1), float64(3), int64(3), object(8)
memory usage: 7.0+ MB


In [231]:
t = merge[merge["_merge"] == "both"]
tt = t[t["발행인유형"] == "시중은행"]

print(
    "Matched Commerical Banks of not matched: ",
    tt.shape, tt.shape[0] / t.shape[0]*100
)

t = merge[merge["_merge"] == "left_only"]
tt = t[t["발행인유형"] == "시중은행"]
print(
    "Only Commerical Banks of not matched: ",
    tt.shape, tt.shape[0] / t.shape[0]*100
)

Matched Commerical Banks of not matched:  (41300, 15) 63.479864740239776
Only Commerical Banks of not matched:  (0, 15) 0.0


In [232]:
merge.tail()

,기준일자,종목번호,종목명,잔존만기,매도유형,매수유형,발행일,만기일,통화,발행금액,발행인유형,금리,할인율,일물,_merge
65141,20241114,KRZE00601E9T,KB국민 자금부 20240927-91-1,1월이상 3월미만,집합투자,증권사,20240927,20241227,KRW,345000000000.00000,시중은행,3.33000,3.51000,3월이상 1년미만,both
65142,20241114,KRZE00601E9T,KB국민 자금부 20240927-91-1,1월이상 3월미만,집합투자,증권사,20240927,20241227,KRW,345000000000.00000,시중은행,3.33000,3.51000,3월이상 1년미만,both
65143,20241114,KRZE00601E9X,KB국민 자금부 20240930-91-1,1월이상 3월미만,집합투자,증권사,20240930,20241230,KRW,145000000000.00000,시중은행,3.33000,3.51000,3월이상 1년미만,both
65144,20241114,KRZE00601E9X,KB국민 자금부 20240930-91-1,1월이상 3월미만,집합투자,증권사,20240930,20241230,KRW,145000000000.00000,시중은행,3.33000,3.51000,3월이상 1년미만,both
65145,20241114,KRZE03701DCD,전북 서울지점 20231213-365-1,7일이상 1월미만,집합투자,증권사,20231213,20241212,KRW,70000000000.00000,지방은행,3.36000,3.94780,3월이상 1년미만,both


In [233]:
print(issue.shape, merge.shape)

issue = issue[issue["발행인유형"] == "시중은행"]
merge = merge[merge["발행인유형"] == "시중은행"]
info_issue_trans(issue, merge, ["통화"])

(3593, 15) (65146, 15)

(1780, 15)
Index(['기준일자', 'ISSUCO_INDTP_CLSF_NO', '발행인유형', 'GOODS_CREDITRT_CD',
       'GOODS_CREDITRT_CD_NM', '통화', '할인율', 'SHORTM_FNCEGD_CD', '종목번호', '종목명',
       '발행일', '만기일', 'GOODS_DAYOBJ_CD', '일물', '발행금액'],
      dtype='object')
issue_date: 20120120 20241115
mat_date: 20120404 20251113
통화 ['KRW']

(41300, 15)
Index(['기준일자', '종목번호', '종목명', '잔존만기', '매도유형', '매수유형', '발행일', '만기일', '통화',
       '발행금액', '발행인유형', '금리', '할인율', '일물', '_merge'],
      dtype='object')
issue_date: 20120120 20241114
mat_date: 20120413 20251113
통화 ['KRW']


# Kofia Merge Completeness
- 수익률과 발행금리는 평균적으로 0.17% 차이나며 
- 28%의 경우 일치하나 23%에서 NEGATIVE, 27%에서 POSITIVE

In [14]:
df = pd.read_csv("kofia\kofia_trans_issue_cd.csv")

In [15]:
df.loc[df["수익률(%)"] >= 40, "수익률(%)"] = None
df["차이"] = (df["수익률(%)"] - df["발행금리(%)"])
print(
    df[df["차이"]<0].shape[0]/df.shape[0], 
    df[df["차이"]==0].shape[0]/df.shape[0], 
    df[df["차이"]>0].shape[0]/df.shape[0]
)
df[["수익률(%)", "발행금리(%)", "차이"]].describe()

0.23314353365655655 0.26926654639756453 0.28014714172962


,수익률(%),발행금리(%),차이
count,70943.00000,55530.00000,55524.00000
mean,2.40307,2.38623,-0.01077
std,1.13558,1.23442,0.37753
min,0.30000,0.55000,-1.80000
25%,1.51000,1.50000,-0.02500
50%,2.27000,1.90000,0.00000
75%,3.52000,3.61000,0.01000
max,8.81000,5.60000,2.54000


In [16]:
df["차이"] = df["차이"].abs()
df[["수익률(%)", "발행금리(%)", "차이"]].describe()

,수익률(%),발행금리(%),차이
count,70943.00000,55530.00000,55524.00000
mean,2.40307,2.38623,0.17078
std,1.13558,1.23442,0.33687
min,0.30000,0.55000,0.00000
25%,1.51000,1.50000,0.00000
50%,2.27000,1.90000,0.01000
75%,3.52000,3.61000,0.14000
max,8.81000,5.60000,2.54000
